In [4]:
import numpy as np
import pandas as pd
import re

In [5]:
# !pip install spacy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

stopwords=list(STOP_WORDS)
from string import punctuation
# !python -m spacy download en_core_web_sm

In [6]:
nlp_spacy = spacy.load('en_core_web_sm')

In [ ]:
pip install nltk

In [7]:
import nltk

In [3]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [53]:
class text_summary:
    def __init__(self, model='nltk'):
        self.model = model
#         from string import punctuation
#         if self.model =='nltk':
#             import nltk
#             nltk.download('punkt')
#         elif self.model =='spacy':
#             import spacy
#             from spacy.lang.en.stop_words import STOP_WORDS
#             stopwords=list(STOP_WORDS)
#             nlp_spacy = spacy.load('en_core_web_sm')
#         else:
#             pass

    
    def clean_words(self, text, remove_stop_words = False):
        """it removes the punctuation and numbers from the text, optional: stop words"""
        if remove_stop_words:
            text = text.split()
#             print(text)
            text = " ".join([x for x in text if x not in stopwords])
        text  = "".join([char for char in text if char not in punctuation])
        text = re.sub('[0-9]+', '', text)   
        text = text.lower().replace('\n', ' ')
        text = re.sub("\s+", " ", text)
#         print(text)
        return text
    
    def clean_data_text(self, df):
        self.data = df
    # also included removing stop words part but not in this case later we will check
        self.data['tokenized_sents'] = self.data['tokenized_sents'].apply(self.clean_words)
    
        return self.data
    

    def create_sent(self, df):

    #     self.data['sentences_list'] = self.data['article_text'].apply(nltk.sent_tokenize)
        self.data = df
        if self.model =='spacy':
            self.data['nlp_docs'] = self.data['article_text'].apply(nlp_spacy)
            # iterating over all the every doc to get the sentecnces
            self.data['sentence_list'] = ''
            for i in range(self.data.shape[0]):
                sentence=  [str(sent) for sent in self.data['nlp_docs'][i].sents]
                self.data.at[self.data.index[i],'sentence_list'] = sentence

        elif self.model == 'nltk':
            self.data['sentence_list'] = self.data['article_text'].apply(nltk.sent_tokenize)
        else:
            pass

        self.data['tokenized_sents'] = self.data['article_text'].astype(str)
        self.data = self.clean_data_text(self.data) # cleaning the article
        self.data['tokenized_sents_copy'] = self.data['tokenized_sents'] #creating copy of sentence without changes
        return self.data
    
    def create_words_frequency_dictionary(self, df):
        self.data = df
        from collections import Counter
        self.data['frequency'] = self.data['tokenized_sents_copy'].str.split(' ').apply(Counter)
        return self.data
    
    def get_maximum_frequency(self, freq_dic):
        return max(freq_dic.values())
    
    def create_word_weighted_freq(self, row_dictionary):
        max_weight = self.get_maximum_frequency(row_dictionary)
        for key, val in row_dictionary.items():
            row_dictionary[key] = val/max_weight
        return row_dictionary
    
#     return max key value pair
#     this is not needed but still created it for future use
    def get_max_key_value(self, freq_dic):
        max_key_value_dic = {}
        k = max(freq_dic, key = freq_dic.get)
        max_key_value_dic[k] = freq_dic[k]
        return max_key_value_dic
    
#     test_function
    def insert_max_word_key_value(self, df):
        self.data = df
        self.data['max'] = self.data['frequency'].apply(self.get_max_key_value)
        return self.data
    
    def calculate_sentence_score(self, sentence_list, 
                             word_weight_dictionary,
                            decided_len=30):
        """This function calculate the sentence score of a sentence by 
        summing up the weighted score of the words. """
        sentence_score = {}
    #     print('step 1')
        for sent in sentence_list:
    #         print(f'sent: {sent}')
            if len(sent.split(' ')) < decided_len:
                for word in sent.lower():
    #                 print(f'{word}')
                    if sent not in sentence_score.keys():

                        sentence_score[sent] = word_weight_dictionary[word]
    #                     print(f'{sentence_score[sent]}')
    #                     print(f'word searched {word}')
    #                     print(f'word_weight_dictionary value First time:{sent} and value\n {word_weight_dictionary[word]}')
                    else:
    #                     print('why this is not printing')
                        sentence_score[sent] += word_weight_dictionary[word]
    #                     print(f'word {word_weight_dictionary[word]} {sentence_score[word]}')
    #                     print(f'word_weight_dictionary value other time time: {sent} and value\n {word_weight_dictionary[word]}')

    #     sentence_score = sorted(sentence_score, key = sentence_score.get, reverse=True)
        return [sentence_score]
    
    def create_summary(self, data, keywords = [], summary_sent_len =7):
        """It calculates the summary on the basis of the highest sentence score.
        It also has a optional keywords parameter which can be used to pass a list 
        of words, if present in any sentence then it will be part of the summary, no matter
        what is the sentence summary."""
        self.df = data
        self.df['sentences_scores_dict'] = 0
        self.df['summary'] = ''
        
        self.df['summary'] = self.df['summary'].astype('O')
        for i in range(self.df.shape[0]):
            dictionary_of_sent_scores = self.calculate_sentence_score(self.df['sentence_list'][i],
                                                                 word_weight_dictionary=self.df['weighted_freq'][i]) 

            
#             self.df.loc[i, 'sentences_scores_dict'] = dictionary_of_sent_scores
            dictionary_of_sent_scores = dictionary_of_sent_scores[0]
            sorted_dict = sorted(dictionary_of_sent_scores,
                                         key = dictionary_of_sent_scores.get,
                                reverse=True)
            
#             if keywords are passed then adding them by increasing the score
#             of those sentences which contains these keywords and also increasing
#             the variable length so that it doesn't skip important sentences
#             on the basis of weights.
#             self.df.at[self.df.index[i], 'summary'] = sorted_dict
#             max_weight = dictionary_of_sent_scores[sorted_dict[0]]
            keywords_sentences_list = [] 
            if len(keywords) > 0:
                
                for k in dictionary_of_sent_scores.keys(): 
                    if any(word in k for word in keywords):
                        keywords_sentences_list.append(k)
    
            
            keywords_sentences_list = keywords_sentences_list + sorted_dict
        
            self.df.loc[i, 'summary'] = " ".join(keywords_sentences_list[:summary_sent_len])

        return self.df
    

In [29]:
df =   df_spacy.copy()

In [30]:
dictionary_of_sent_scores = spacy_nlp_model.calculate_sentence_score(df['sentence_list'][0],
                                                     word_weight_dictionary=df['weighted_freq'][0]) 
print(dictionary_of_sent_scores)

[{'TCS a Leader in System Integrator Capabilities on Microsoft Azure:': 3.0, 'Everest Group\nTata Consultancy Services Leverages its Edge to Cloud Technology Expertise on Azure, Deep Contextual and Domain Knowledge, and Comprehensive Portfolio of Industry Solutions to Drive Market Success\n': 5.0, '\n': 0, 'The report highlights TCS\x92 industry-focused next-generation solutions on Azure, spanning edge, Internet of Things, blockchain, and AI-/ ML-based solutions, as key strengths.': 4.0, 'Also highlighted as strengths are the company\x92s vast library of assets, blueprints, and accelerators to help automate different stages of the cloud journey on Azure.': 7.0, 'According to the report, clients have appreciated TCS\x92 partnership with Microsoft, competencies on Azure platforms, and end-to-end capabilities entailing consulting, design/ build, modernization, and operate services.': 7.0, '\x93We believe that enterprises will harness the native capabilities of the cloud to embrace innovat

In [31]:
dictionary_of_sent_scores[0]

{'TCS a Leader in System Integrator Capabilities on Microsoft Azure:': 3.0,
 'Everest Group\nTata Consultancy Services Leverages its Edge to Cloud Technology Expertise on Azure, Deep Contextual and Domain Knowledge, and Comprehensive Portfolio of Industry Solutions to Drive Market Success\n': 5.0,
 '\n': 0,
 'The report highlights TCS\x92 industry-focused next-generation solutions on Azure, spanning edge, Internet of Things, blockchain, and AI-/ ML-based solutions, as key strengths.': 4.0,
 'Also highlighted as strengths are the company\x92s vast library of assets, blueprints, and accelerators to help automate different stages of the cloud journey on Azure.': 7.0,
 'According to the report, clients have appreciated TCS\x92 partnership with Microsoft, competencies on Azure platforms, and end-to-end capabilities entailing consulting, design/ build, modernization, and operate services.': 7.0,
 '\x93We believe that enterprises will harness the native capabilities of the cloud to embrace in

In [32]:
#             df.loc[i, 'sentences_scores_dict'] = dictionary_of_sent_scores
dictionary_of_sent_scores =dictionary_of_sent_scores[0]

In [33]:
sorted_dict = sorted(dictionary_of_sent_scores,
                             key = dictionary_of_sent_scores.get,
                    reverse=True)
print(sorted_dict)

['Several of TCS\x92 proprietary products and platforms are available on Azure, including the TCS Industrial Power Plant (IP2\x99) Solution that leverages the Azure Digital TWIN and', 'Also highlighted as strengths are the company\x92s vast library of assets, blueprints, and accelerators to help automate different stages of the cloud journey on Azure.', 'According to the report, clients have appreciated TCS\x92 partnership with Microsoft, competencies on Azure platforms, and end-to-end capabilities entailing consulting, design/ build, modernization, and operate services.', 'TCS provides end-to-end application modernization and migration services on Azure, using a customer-centric, productized approach that accelerates the transformation journey.', 'Our best-in-class and future-ready edge to cloud business solutions on Azure help our customers accelerate their multi-horizon cloud adoption journeys,\x94 said Siva Ganesan, Global Head, Microsoft Business Unit, TCS.', 'As a result, TCS has

In [35]:
#             if keywords are passed then adding them by increasing the score
#             of those sentences which contains these keywords and also increasing
#             the variable length so that it doesn't skip important sentences
#             on the basis of weights.
#             df.at[df.index[0], 'summary'] = sorted_dict
#             max_weight = dictionary_of_sent_scores[sorted_dict[0]]
keywords_sentences_list = [] 
keywords = ['Market']
summary_sent_len = 7

In [36]:
if len(keywords) > 0:

    for k in dictionary_of_sent_scores.keys(): 
        if any(word in k for word in keywords):
            keywords_sentences_list.append(k)

In [39]:
keywords_sentences_list

['Everest Group\nTata Consultancy Services Leverages its Edge to Cloud Technology Expertise on Azure, Deep Contextual and Domain Knowledge, and Comprehensive Portfolio of Industry Solutions to Drive Market Success\n',
 'Several of TCS\x92 proprietary products and platforms are available on Azure, including the TCS Industrial Power Plant (IP2\x99) Solution that leverages the Azure Digital TWIN and',
 'Also highlighted as strengths are the company\x92s vast library of assets, blueprints, and accelerators to help automate different stages of the cloud journey on Azure.',
 'According to the report, clients have appreciated TCS\x92 partnership with Microsoft, competencies on Azure platforms, and end-to-end capabilities entailing consulting, design/ build, modernization, and operate services.',
 'TCS provides end-to-end application modernization and migration services on Azure, using a customer-centric, productized approach that accelerates the transformation journey.',
 'Our best-in-class a

In [38]:
keywords_sentences_list = keywords_sentences_list + sorted_dict

In [51]:
df.loc[i, 'summary'] = " ".join(list(keywords_sentences_list)[:7])

In [52]:
df.loc[i, 'summary']

'Everest Group\nTata Consultancy Services Leverages its Edge to Cloud Technology Expertise on Azure, Deep Contextual and Domain Knowledge, and Comprehensive Portfolio of Industry Solutions to Drive Market Success\n Several of TCS\x92 proprietary products and platforms are available on Azure, including the TCS Industrial Power Plant (IP2\x99) Solution that leverages the Azure Digital TWIN and Also highlighted as strengths are the company\x92s vast library of assets, blueprints, and accelerators to help automate different stages of the cloud journey on Azure. According to the report, clients have appreciated TCS\x92 partnership with Microsoft, competencies on Azure platforms, and end-to-end capabilities entailing consulting, design/ build, modernization, and operate services. TCS provides end-to-end application modernization and migration services on Azure, using a customer-centric, productized approach that accelerates the transformation journey. Our best-in-class and future-ready edge 

In [5]:
t = text_summary(model='nltk')

In [8]:
t.clean_words(text='anuj               not a is   88979good    *&BOY', remove_stop_words=True)

'anuj good boy'

In [54]:
import pandas as pd
if __name__ == "__main__":
#     loading data
    df = pd.read_csv('tennis_articles.csv',encoding = 'unicode_escape')
#     creating copy of data
    df_nltk = df.copy()
    df_spacy = df.copy()
#     creating instances of class
    spacy_nlp_model = text_summary(model='spacy')
    nltk_nlp_model = text_summary(model='nltk')
#     creating sentences
    df_spacy = spacy_nlp_model.create_sent(df_spacy)
    df_nltk = nltk_nlp_model.create_sent(df_nltk)
#     creating word dictionaries
    df_spacy = spacy_nlp_model.create_words_frequency_dictionary(df_spacy)
    df_nltk = nltk_nlp_model.create_words_frequency_dictionary(df_nltk)
#     It is a test function
    df_spacy = spacy_nlp_model.insert_max_word_key_value(df_spacy)
    df_nltk = nltk_nlp_model.insert_max_word_key_value(df_nltk)

    
# this function has one bug, it is changing the main column of frequency also.
    df_spacy['weighted_freq'] = df_spacy['frequency'].apply(spacy_nlp_model.create_word_weighted_freq)
    df_nltk['weighted_freq'] = df_nltk['frequency'].apply(nltk_nlp_model.create_word_weighted_freq)
    
    
#     for spacy first  
    df_spacy = spacy_nlp_model.create_summary(df_spacy)
#     for nltk 
    df_nltk = nltk_nlp_model.create_summary(df_nltk)
#     saving the ouput in csv file
    df_spacy.to_csv('spacy_summary.csv')
    df_nltk.to_csv('nltk_summary.csv')

In [23]:
    
    df_spacy = spacy_nlp_model.create_summary(df_spacy)
#     for nltk 
    df_nltk = nltk_nlp_model.create_summary(df_nltk)
    
    

    
    

In [55]:
df_spacy

,article_id,article_title,article_text,source,nlp_docs,sentence_list,tokenized_sents,tokenized_sents_copy,frequency,max,weighted_freq,sentences_scores_dict,summary
0,1,TCS a Leader in System Integrator Capabilities...,TCS a Leader in System Integrator Capabilities...,https://www.tcs.com/leader-system-integrator-c...,"(TCS, a, Leader, in, System, Integrator, Capab...",[TCS a Leader in System Integrator Capabilitie...,tcs a leader in system integrator capabilities...,tcs a leader in system integrator capabilities...,"{'tcs': 0.36666666666666664, 'a': 0.5, 'leader...",{'and': 30},"{'tcs': 0.36666666666666664, 'a': 0.5, 'leader...",0,Several of TCS proprietary products and platf...
1,2,OptumInsight and Change Healthcare Combine to ...,"Optum, a diversified health services company a...",https://www.optum.com/about-us/news/optuminsig...,"(Optum, ,, a, diversified, health, services, c...","[Optum, a diversified health services company ...",optum a diversified health services company an...,optum a diversified health services company an...,"{'optum': 0.10204081632653061, 'a': 0.22448979...",{'and': 49},"{'optum': 0.10204081632653061, 'a': 0.22448979...",0,"Change Healthcare brings key technologies, con..."


In [56]:
df_nltk.head()

,article_id,article_title,article_text,source,sentence_list,tokenized_sents,tokenized_sents_copy,frequency,max,weighted_freq,sentences_scores_dict,summary
0,1,TCS a Leader in System Integrator Capabilities...,TCS a Leader in System Integrator Capabilities...,https://www.tcs.com/leader-system-integrator-c...,[TCS a Leader in System Integrator Capabilitie...,tcs a leader in system integrator capabilities...,tcs a leader in system integrator capabilities...,"{'tcs': 0.36666666666666664, 'a': 0.5, 'leader...",{'and': 30},"{'tcs': 0.36666666666666664, 'a': 0.5, 'leader...",0,Also highlighted as strengths are the company...
1,2,OptumInsight and Change Healthcare Combine to ...,"Optum, a diversified health services company a...",https://www.optum.com/about-us/news/optuminsig...,"[Optum, a diversified health services company ...",optum a diversified health services company an...,optum a diversified health services company an...,"{'optum': 0.10204081632653061, 'a': 0.22448979...",{'and': 49},"{'optum': 0.10204081632653061, 'a': 0.22448979...",0,"Change Healthcare brings key technologies, con..."


In [59]:
df_spacy.to_csv('spacy_summary.csv')